# Lab 3

Link del respositorio: https://github.com/faguilarleal/lab3_vision 

#### Integrantes:
 - Francis Aguilar
 - César Lopez
 - Jose Marchena

# Task 2

<!-- En esta parte se busca que usted implemente un sistema de correspondencia completo. Para esto debe -->
escribir un script en Python usando OpenCV. No se provee código base, debe estructurarlo usted mismx.
Para esta parte debe crear su propia imagen a usar para esto:
• Tome dos fotografías propias de un objeto con textura (i.e. una caja de cereral, una portada de
libro, un edificio)
• Foto 1: Vista frontal
• Foto 2: Vista rotada (aproximadamente 45 grados) y con cambio de escala (aléjese o haga zoom).
El cambio debe ser evidente
Con esto haga lo siguiente en su código
1. Cargue ambas imágenes en escala de grises
2. Implemente la detección y descripción usando SIFT.
3. Implemente la detección y descripción usando ORB.

In [2]:
import cv2
import numpy as np

ruta_img1 = "foto_frontal.jpg"
ruta_img2 = "foto_rotada.jpg"

img1 = cv2.imread(ruta_img1)
img2 = cv2.imread(ruta_img2)

if img1 is None or img2 is None:
    raise IOError("Error cargando una o ambas imágenes")

img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


OSError: Error cargando una o ambas imágenes

In [ ]:
sift = cv2.SIFT_create()

kp1_sift, des1_sift = sift.detectAndCompute(img1_gray, None)
kp2_sift, des2_sift = sift.detectAndCompute(img2_gray, None)

print("SIFT")
print("Imagen 1 - Keypoints:", len(kp1_sift))
print("Imagen 2 - Keypoints:", len(kp2_sift))
print("Descriptor img1:", des1_sift.shape if des1_sift is not None else None)
print("Descriptor img2:", des2_sift.shape if des2_sift is not None else None)


In [ ]:
orb = cv2.ORB_create(nfeatures=5000)

kp1_orb, des1_orb = orb.detectAndCompute(img1_gray, None)
kp2_orb, des2_orb = orb.detectAndCompute(img2_gray, None)

print("ORB")
print("Imagen 1 - Keypoints:", len(kp1_orb))
print("Imagen 2 - Keypoints:", len(kp2_orb))
print("Descriptor img1:", des1_orb.shape if des1_orb is not None else None)
print("Descriptor img2:", des2_orb.shape if des2_orb is not None else None)


In [ ]:
# BFMatcher para SIFT (norma L2)
bf_sift = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# KNN matching (k=2 para Lowe)
matches_sift = bf_sift.knnMatch(des1_sift, des2_sift, k=2)

# Lowe's Ratio Test
good_matches_sift = []
ratio_thresh = 0.75

for m, n in matches_sift:
    if m.distance < ratio_thresh * n.distance:
        good_matches_sift.append(m)

print("SIFT Matching")
print("Matches totales:", len(matches_sift))
print("Good matches:", len(good_matches_sift))


In [ ]:
# BFMatcher para ORB (norma Hamming)
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

# KNN matching
matches_orb = bf_orb.knnMatch(des1_orb, des2_orb, k=2)

# Lowe's Ratio Test
good_matches_orb = []

for m, n in matches_orb:
    if m.distance < ratio_thresh * n.distance:
        good_matches_orb.append(m)

print("ORB Matching")
print("Matches totales:", len(matches_orb))
print("Good matches:", len(good_matches_orb))


In [ ]:
img_matches_sift = cv2.drawMatches(
    img1,
    kp1_sift,
    img2,
    kp2_sift,
    good_matches_sift,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

# Mostrar en Jupyter
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
plt.imshow(cv2.cvtColor(img_matches_sift, cv2.COLOR_BGR2RGB))
plt.title("SIFT - Good Matches (Lowe Ratio Test)")
plt.axis("off")
plt.show()


In [ ]:
img_matches_orb = cv2.drawMatches(
    img1,
    kp1_orb,
    img2,
    kp2_orb,
    good_matches_orb,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

plt.figure(figsize=(15, 8))
plt.imshow(cv2.cvtColor(img_matches_orb, cv2.COLOR_BGR2RGB))
plt.title("ORB - Good Matches (Lowe Ratio Test)")
plt.axis("off")
plt.show()
